In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import io
import numpy as np

Using TensorFlow backend.


In [3]:
from google.colab import files
uploaded = files.upload()

Saving lyric_data.txt to lyric_data (1).txt


In [0]:
with open('lyric_data.txt', 'r', encoding='utf-8') as f:
    corpus = f.read()

In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for song in corpus.split("@@@"):
    _, lyrics = song.split("|||")
    lines = lyrics.split('\n')
    for i in range(len(lines)-1):
      if len(lines[i])<2 or len(lines[i+1])<2:
        continue
      if lines[i][0] != '[':
        input_texts.append(lines[i])
        target_texts.append('†' + lines[i+1] + '‡')
        for char in input_texts[-1]:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_texts[-1]:
            if char not in target_characters:
                target_characters.add(char)

In [0]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
num_samples = len(input_texts)

In [7]:
print('Number of samples:', num_samples)
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 19837
Number of unique input tokens: 96
Number of unique output tokens: 102
Max sequence length for inputs: 140
Max sequence length for outputs: 142


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
batch_size = 100  # Batch size for training.
epochs = 25  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = int(num_samples*0.7)  # Number of samples to train on.

In [12]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 15869 samples, validate on 3968 samples
Epoch 1/25
15869/15869 [==============================] - 74s 5ms/step - loss: 0.8562 - val_loss: 0.7877
Epoch 2/25
15869/15869 [==============================] - 72s 5ms/step - loss: 0.7013 - val_loss: 0.6762
Epoch 3/25
15869/15869 [==============================] - 72s 5ms/step - loss: 0.6263 - val_loss: 0.6286
Epoch 4/25
15869/15869 [==============================] - 73s 5ms/step - loss: 0.5838 - val_loss: 0.5967
Epoch 5/25
15869/15869 [==============================] - 72s 5ms/step - loss: 0.5534 - val_loss: 0.5752
Epoch 6/25
15869/15869 [==============================] - 72s 5ms/step - loss: 0.5293 - val_loss: 0.5527
Epoch 7/25
15869/15869 [==============================] - 72s 5ms/step - loss: 0.5103 - val_loss: 0.5372
Epoch 8/25
15869/15869 [==============================] - 71s 4ms/step - loss: 0.4932 - val_loss: 0.5244
Epoch 9/25
158

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['†']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '‡' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [30]:
sample = "Swag on 100"
inp = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
for t, char in enumerate(sample):
    inp[0, t, input_token_index[char]] = 1.
print(decode_sequence(inp))

I got the dope in my sock‡
